In [4]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF

def extract_labels(labels):
    # Map emotion labels to integers
    label_mapping = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
    return np.array([label_mapping[label] for label in labels])

class EmotionDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.data, self.labels = self.load_data(dataset_path)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
    @staticmethod
    def load_data(dataset_path):
        data = []
        labels = []

        for emotion_folder in os.listdir(dataset_path):
            emotion_path = os.path.join(dataset_path, emotion_folder)
            for img_name in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path)
                data.append(img)
                labels.append(emotion_folder)

        return np.array(data), extract_labels(labels)
    
    


In [5]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
train_path = 'FER2013/train'
absolute_train_path = os.path.abspath(train_path)

test_path = 'FER2013/test'
absolute_test_path = os.path.abspath(test_path)

train_dataset = EmotionDataset(absolute_train_path, transform=transform)
test_dataset = EmotionDataset(absolute_test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Load pre-trained model

In [11]:
# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda") # Use the GPU for computation
    print("Using GPU for computation!")
else:
    device = torch.device("cpu") # Use the CPU for computation
    print("Using CPU for computation!")

Using GPU for computation!


In [12]:
from torchvision import transforms, models

model = models.resnet18(pretrained=True)

# Modify the final layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 7)  

In [13]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
if torch.cuda.is_available():
    model.to('cuda')
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to('cuda')
else:
    device = torch.device('cpu')
print('Using device:', device)

Using device: cuda


Define Loss function and optimiztion

In [22]:
# Modify the loss function and optimizer to work with GPU
criterion = criterion.to(device)

In [34]:
num_epochs = 10

for epoch in range(num_epochs):

    model.train()
    
    train_accuracy = 0
    num_correct = 0
    num_samples = 0
    
    for inputs, labels in train_loader: 
     
        optimizer.zero_grad()
        
        # Forward pass
        inputs = inputs.to(device) 
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        train_accuracy += acc
        
    # Print epoch loss and accuracy
    avg_acc = train_accuracy / len(train_loader)            
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {avg_acc:.4f}')

Epoch 1/10, Loss: 0.0078, Accuracy: 0.9645
Epoch 2/10, Loss: 0.1049, Accuracy: 0.9688
Epoch 3/10, Loss: 0.0711, Accuracy: 0.9668
Epoch 4/10, Loss: 0.0454, Accuracy: 0.9730
Epoch 5/10, Loss: 0.0203, Accuracy: 0.9752
Epoch 6/10, Loss: 0.0038, Accuracy: 0.9795
Epoch 7/10, Loss: 0.0009, Accuracy: 0.9790
Epoch 8/10, Loss: 0.2897, Accuracy: 0.9795
Epoch 9/10, Loss: 0.0001, Accuracy: 0.9769
Epoch 10/10, Loss: 0.2857, Accuracy: 0.9837


In [33]:
for inputs, labels in train_loader:
    print(type(inputs))
    print(type(labels))
    print(inputs)
    print(labels)
    break

<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([[[[ 1.5810,  1.5810,  1.5810,  ...,  0.2624,  0.2624,  0.2624],
          [ 1.5810,  1.5810,  1.5810,  ...,  0.2624,  0.2624,  0.2624],
          [ 1.5810,  1.5810,  1.5810,  ...,  0.2624,  0.2624,  0.2624],
          ...,
          [-0.1657, -0.1657, -0.1657,  ..., -1.8782, -1.8782, -1.8782],
          [-0.1828, -0.1828, -0.1828,  ..., -1.8782, -1.8782, -1.8782],
          [-0.1828, -0.1828, -0.1828,  ..., -1.8782, -1.8782, -1.8782]],

         [[ 1.7458,  1.7458,  1.7458,  ...,  0.3978,  0.3978,  0.3978],
          [ 1.7458,  1.7458,  1.7458,  ...,  0.3978,  0.3978,  0.3978],
          [ 1.7458,  1.7458,  1.7458,  ...,  0.3978,  0.3978,  0.3978],
          ...,
          [-0.0399, -0.0399, -0.0399,  ..., -1.7906, -1.7906, -1.7906],
          [-0.0574, -0.0574, -0.0574,  ..., -1.7906, -1.7906, -1.7906],
          [-0.0574, -0.0574, -0.0574,  ..., -1.7906, -1.7906, -1.7906]],

         [[ 1.9603,  1.9603,  1.9603,  ...,  0.6182,  0.

In [35]:
model.eval()
test_accuracy = 0  
num_correct = 0
num_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        test_accuracy += acc

print(f'Test Accuracy: {test_accuracy / len(test_loader):.4f}')

Test Accuracy: 0.6262
